In [1]:
# using Pkg
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
# Pkg.add("Distances")
# Pkg.add("Distributions")
# Pkg.add("DataFrames")

In [2]:
using JuMP, Gurobi, Graphs, Plots, StatsPlots, DataFrames, Random, Printf, LinearAlgebra, Distributions, Distances, Suppressor
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18


Gurobi.Env(Ptr{Nothing} @0x00000001365e5800, false, 0)

# Generate Data

## Constants

In [28]:
struct Warehouse #supply
    grid_size::Float64
    time_horizon::Int
    distance_to_arrival::Vector{Float64}
    distance_to_departure::Vector{Float64}
    capacity::Vector{Float64}
end

struct Shipment #demand
    arrival_time::Vector
    departure_time::Vector
    shipment_size::Vector 
end

function ShipmentData(time_horizon, num_shipment, max_shipment_size)
    rand_shipment_time = sort(rand(1:T, (num_shipment, 2)), dims = 2)
    rand_sizes = rand(1:max_shipment_size, num_shipment)

    return Shipment(rand_shipment_time[:, 1], rand_shipment_time[:, 2], rand_sizes)
end

function WarehouseData(grid_size, time_horizon, warehouse_row, warehouse_col, max_shipment_size, cap_tightness)
    arrival_location = (0, 0)
    departure_location = (warehouse_row, 0)

    location = [(i*grid_size,j*grid_size) for i =1:warehouse_row, j = 1:warehouse_col]
    L = warehouse_row * warehouse_col
    distance_to_arrival = [norm(arrival_location .- location[l]) for l in 1:L]
    distance_to_departure = [norm(arrival_location .- location[l]) for l in 1:L]
    #cap_tightness \in [0,1]: 0 is relatively tight (uniform), 1 is null size capacity
    capacity = rand(floor(max_shipment_size * cap_tightness):max_shipment_size, warehouse_row * warehouse_col) 
    return Warehouse(
        grid_size,
        time_horizon,
        distance_to_arrival,
        distance_to_departure,
        capacity
    )
end

WarehouseData (generic function with 1 method)

# Build Model

demand packs one supply (occupancy) and size of demand packs supply size (size) 

\begin{align}
\sum_{i \in 1:I} x_{ilt} \leq 1 \quad \text{occupancy (shipment1, shipment2, location)}\\
\sum_{i \in 1:I} d_i x_i \leq s \quad (\text{vs} d_i x_i \leq s \forall i )\quad\text{size (shipment1, location)}\\ 

\\
\sum_{i \in 1:I} x_{ij} \leq 1 \forall j \quad\text{occupancy}\\
\sum_{i \in 1:I} d_i x_{ij} \leq s_j \forall j \quad\text{size}\\
\end{align}

In [29]:
function three_slt_extended_time_space(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);

    @variable(model, r[1:S,1:L,1:T] >= 0, Bin);
    @variable(model, z[1:S,1:L] >= 0, Bin);

    # supply side: 
    # can't store more than one shipment at each location and time
    @constraint(model, occupancy_pack[l in 1:L, t in 1:T],
        sum(r[s,l,t] for s in 1:S) <=1)
    
    # Each location can only hold items up to max size of the location
    @constraint(model, size_pack[l in 1:L, t in 1:T],
        sum(r[s,l,t] * shipment_data.shipment_size[s] for s in 1:S) <= warehouse_data.capacity[l] ## TODO sum over shipment or not
    );

    # Force z[s,l] to capture the location we're storing s in
    @constraint(model, no_bumping[s in 1:S, l in 1:L, t in shipment_data.arrival_time[s]:shipment_data.departure_time[s]], 
        r[s,l,t] == z[s,l])

    # Each shipment must be assigned to exactly one location during its time window:
    @constraint(model, demand_pack_supply_once[s in 1:S, t in shipment_data.arrival_time[s]:shipment_data.departure_time[s]],
        sum(r[s,l,t] for l in 1:L) == 1)

    # Each shipment must be assigned to exactly zero locations outside its time window:
    @constraint(model, mute_before_timewindow[s in 1:S, t in 1:shipment_data.arrival_time[s]-1],
        sum(r[s,l,t] for l in 1:L) == 0)

    @constraint(model, mute_after_timewindow[s in 1:S, t in shipment_data.departure_time[s]+1:T],
        sum(r[s,l,t] for l in 1:L) == 0)

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) / (shipment_data.departure_time[s] - shipment_data.arrival_time[s] + 1) * r[s,l,t] 
        for s in 1:S, l in 1:L,t in 1:T));

    return model, r, z
end


three_slt_extended_time_space (generic function with 2 methods)

In [30]:
function two_sl_so(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);

    @variable(model, z[1:S,1:L] >= 0, Bin);

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) * z[s,l] for s in 1:S, l in 1:L))


    # Each product must be assigned to exactly one location:
    @constraint(model, location_assignment[s in 1:S],
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );

    # Each location can only hold items up to max size of the location
    @constraint(model, max_size[l in 1:L, s in 1:S],
        z[s,l] * shipment_data.shipment_size[s] <= warehouse_data.capacity[l]
    );

    # Each location can hold at most one product at any specific time:
    overlap_dict = Dict{Int64,Vector{Int64}}()
    for s1 in 1:S
        s = []
        for s2 in s1+1:S
            if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
                append!(s, s2)
            end
        end
        overlap_dict[s1] = s
    end

    # # Each location can hold at most one product at any specific time:
    # function find_overlap(s1)
    #     s = []
    #     for s2 in s1+1:S
    #         if s1 == s2
    #             continue
    #         end

    #         if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
    #             append!(s, s2)
    #         end
    #     end
    #     return s
    # end

    # @constraint(model, no_overlap[s1 in 1:S, s2 in find_overlap(s1), l in 1:L],
    #     z[s1,l] + z[s2,l] <= 1
    # );
    @constraint(model, no_overlap[s1 in 1:S, s2 in overlap_dict[s1], l in 1:L],
        z[s1,l] + z[s2,l] <= 1
    );

    return model, z
end


two_sl_so (generic function with 2 methods)

In [31]:
function two_sl_cso(
    warehouse_data::Warehouse,
    shipment_data::Shipment
)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60);

    @variable(model, z[1:S,1:L] >= 0, Bin);

    @variable(
        model,
        1 <= r[1:S] <= L, Int
    ) ;

    @variable(
        model,
        y[1:S, 1:S], Bin
    ) ;

    @objective(model, Min, 
        sum((warehouse_data.distance_to_arrival[l] + warehouse_data.distance_to_departure[l]) * z[s,l] for s in 1:S, l in 1:L))

    # Make r[s] equal to the index of the location we're storing s in
    @constraint(model, assign_location_index[s in 1:S],
        sum(
            l * z[s,l]
            for l in 1:L
        ) == r[s]
    );

    # Each product must be assigned to exactly one location:
    @constraint(model, location_assignment[s in 1:S],
        sum(
            z[s,l]
            for l in 1:L
        ) == 1
    );

    # Each location can only hold items up to max size of the location
    @constraint(model, max_size[l in 1:L, s in 1:S],
        z[s,l] * shipment_data.shipment_size[s] <= warehouse_data.capacity[l] ## TODO sum over shipment or not
    );

    # Each location can hold at most one product at any specific time:
    overlap_dict = Dict{Int64,Vector{Int64}}()
    for s1 in 1:S
        s = []
        for s2 in s1+1:S
            if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
                append!(s, s2)
            end
        end
        overlap_dict[s1] = s
    end

    # function find_overlap(s1)
    #     s = []
    #     for s2 in s1:S
    #         if shipment_data.arrival_time[s1] <= shipment_data.departure_time[s2] && shipment_data.arrival_time[s2] <= shipment_data.departure_time[s1]
    #             append!(s, s2)
    #         end
    #     end
    #     return s
    # end

    # @constraint(model, no_overlap_1[s1 in 1:S, s2 in find_overlap(s1)],
    #     r[s1] - r[s2] <= -.01 + (L+5) * y[s1,s2]
    # );
    # @constraint(model, no_overlap_2[s1 in 1:S, s2 in find_overlap(s1)],
    #     r[s1] - r[s2] >= .01 - (1-y[s1,s2]) * (L+5)
    # );
    @constraint(model, no_overlap_1[s1 in 1:S, s2 in overlap_dict[s1]],
        r[s1] - r[s2] <= -.01 + (L+5) * y[s1,s2]
    );
    @constraint(model, no_overlap_2[s1 in 1:S, s2 in overlap_dict[s1]],
        r[s1] - r[s2] >= .01 - (1-y[s1,s2]) * (L+5)
    );

    return model, z
end


two_sl_cso (generic function with 2 methods)

In [32]:
#findall(x->x==1, value.(z[1,:]))
# findall(x->x==1, value.(r[1,:,:]))

In [33]:
G = 2
R = 2
C = 2
L = R*C
T = 2
max_shipment_size = 10
S = 2
cap_tightness = .5
WarehouseData(G,  T, R, C, max_shipment_size, cap_tightness)
ShipmentData(T, S, max_shipment_size)

Shipment([2, 1], [2, 1], [8, 7])

In [34]:
shipment_data =ShipmentData(T, S, max_shipment_size)

Shipment([1, 2], [2, 2], [10, 1])

In [35]:
shipment_data

Shipment([1, 2], [2, 2], [10, 1])

In [37]:
Random.seed!(3000)

plotdata = []
# packing S paths in LT space
L_range = [(20, 20)] #, (30, 75)]
S_range = [50, 100] #, 100, 250]
T_range = [10, 20]
cap_tightness_range = [0.5, 1]
for ((r, c), s, t, cap_tightness) in Iterators.product(
    L_range,
    S_range,
    T_range,
    cap_tightness_range
)
    G = 2
    R = r
    C = c
    L = R*C
    T = t
    max_shipment_size = 10
    S = s
# #function WarehouseData(grid_size, warehouse_row, warehouse_col, max_shipment_size, time_horizon, cap_tightness)
    warehouse_data = WarehouseData(G, T, R, C, max_shipment_size, cap_tightness)
    shipment_data = ShipmentData(T, S, max_shipment_size)

    @printf "Starting 1\n"
    buildtime1 = @elapsed model1, _, _ = three_slt_extended_time_space(warehouse_data, shipment_data)
    # optimize!(model1)
    solvetime1 = @elapsed @suppress optimize!(model1)
    objective1 = typemax(Int32)
    try
        objective1 = objective_value(model1)
    catch
    end

    @printf "Starting 2\n"
    buildtime2 = @elapsed model2, _ = two_sl_so(warehouse_data, shipment_data)
    # optimize!(model2)
    solvetime2 = @elapsed @suppress optimize!(model2)
    objective2 = typemax(Int32)
    try
        objective2 = objective_value(model2)
    catch
    end
    
    # @printf "Starting 3\n"
    # buildtime3 = @elapsed model3, _ = two_sl_cso(warehouse_data, shipment_data)
    # # optimize!(model3)
    # solvetime3 = @elapsed @suppress optimize!(model3)
    # objective3 = typemax(Int32)
    # try
    #     objective3 = objective_value(model3)
    # catch
    # end

    push!(plotdata, (
        L = L,
        S = S,
        T = T,
        buildtime1 = buildtime1, 
        solvetime1 = solvetime1, 
        objective1 = objective1,
        buildtime2 = buildtime2, 
        solvetime2 = solvetime2, 
        objective2 = objective2,
        # buildtime3 = buildtime3, 
        # solvetime3 = solvetime3,
        # objective3 = objective3,
    ))
end
df = DataFrame(plotdata)

@show df

Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


...

Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


Starting 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter TimeLimit to value 60


df = 8×9 DataFrame
 Row │ L      S      T      buildtime1  solvetime1  objective1  buildtime2  solvetime2  objective2
     │ Int64  Int64  Int64  Float64     Float64     Float64     Float64     Float64     Float64
─────┼─────────────────────────────────────────────────────────────────────────────────────────────
   1 │   400     50     10    1.75247     2.13213      799.253    1.23439     0.937139     799.253
   2 │   400    100     10    1.0036      1.63554     2085.1      3.38351     3.98574     2085.1
   3 │   400     50     20    1.31537     1.81491      840.542    0.500866    1.07226      840.542
   4 │   400    100     20    1.35096     3.73144     1941.59     3.01552     4.98701     1941.59
   5 │   400     50     10    0.651577    0.827475     831.619    0.797648    0.957169     831.619
   6 │   400    100     10    1.25648     1.38029     2244.19     2.87043     5.0044      2244.19
   7 │   400     50     20    0.60778     3.05495      756.825    1.03853     0.998701     756.8

Row,L,S,T,buildtime1,solvetime1,objective1,buildtime2,solvetime2,objective2
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,400,50,10,1.75247,2.13213,799.253,1.23439,0.937139,799.253
2,400,100,10,1.0036,1.63554,2085.1,3.38351,3.98574,2085.1
3,400,50,20,1.31537,1.81491,840.542,0.500866,1.07226,840.542
4,400,100,20,1.35096,3.73144,1941.59,3.01552,4.98701,1941.59
5,400,50,10,0.651577,0.827475,831.619,0.797648,0.957169,831.619
6,400,100,10,1.25648,1.38029,2244.19,2.87043,5.0044,2244.19
7,400,50,20,0.60778,3.05495,756.825,1.03853,0.998701,756.825
8,400,100,20,2.23797,98.9833,2057.78,3.01197,5.28922,2057.78


In [11]:
groupedbar(
    repeat(lpad.(string.(L_range), 4), outer = 3), 
    vcat(df[!, :solvetime1], df[!, :solvetime2], df[!, :solvetime3]),
    group = repeat(["Model 1", "Model 2", "Model 3"], inner = 3),
    ylabel = "Solve time (s)",
    # yscale = :log10,
    title = "Solve times of both formulations with # of customers",
    legend = :topleft,
)

UndefVarError: UndefVarError: df not defined

In [12]:
df = DataFrame(plotdata)
@show df

df = 0×0 DataFrame


0×0 DataFrame

In [13]:
df = DataFrame(plotdata)
@show df

df = 0×0 DataFrame


0×0 DataFrame